## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [ ]:
import numba
from numba import jit, njit

In [ ]:
import numpy as np
a= np.random.randint(0,1000,10**6)
b = [i+100 for i in a]
print(np.mean(b))

599.387236


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [ ]:
import pandas as pd
import numpy as np
import string
import random
df = pd.DataFrame(np.random.randint(100,size=(2*10**6, 4)))
df["key"] = [random.choice(string.ascii_lowercase) for i in range(2*10**6)]
options = ["a", "b","c", "d","e"]
df
rslt_df = df[df['key'].isin(options)]
rslt_df

,0,1,2,3,key
0,18,6,76,71,c
10,7,20,33,74,e
24,54,38,18,36,c
34,64,97,92,3,a
38,37,37,42,89,a
...,...,...,...,...,...
1999992,96,9,33,38,a
1999993,56,82,3,62,b
1999995,47,83,12,70,b
1999998,57,11,50,58,d


## Лабораторная работа 3

In [ ]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 10.3 MB/s eta 0:00:00


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [ ]:
import pandas as pd
rec = pd.read_csv('recipes_sample.csv')
rev = pd.read_csv("reviews_sample.csv")
#А
%time
a1 = []
for i, row in rev[(rev['date']<"2011-01-01") & (rev["date"]>= "2010-01-01")].iterrows():
  a1.append(row["rating"])
print(sum(a1)/len(a1))
#Б
%time
revr = rev[(rev['date']<"2011-01-01") & (rev["date"]>= "2010-01-01")]
a2 = []
for i, row in revr.iterrows():
  a2.append(row["rating"])
print(sum(a2)/len(a2))
#В
%time
print(revr["rating"].mean(skipna = True))



CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 13.8 µs
4.4544402182900615
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
4.4544402182900615
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.68 µs
4.4544402182900615


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [ ]:
from line_profiler import LineProfiler
def A():
  a1 = []
  for i, row in rev[(rev['date']<"2011-01-01") & (rev["date"]>= "2010-01-01")].iterrows():
    a1.append(row["rating"])
  print(sum(a1)/len(a1))

def B():
  revr = rev[(rev['date']<"2011-01-01") & (rev["date"]>= "2010-01-01")]
  a2 = []
  for i, row in revr.iterrows():
    a2.append(row["rating"])
  print(sum(a2)/len(a2))

def V():
  print(revr["rating"].mean(skipna = True))


lprofiler1 = LineProfiler()
lprofiler2 = LineProfiler()
lprofiler3 = LineProfiler()

lp_wrapper1 = lprofiler1(A)
lp_wrapper2 = lprofiler2(B)
lp_wrapper3 = lprofiler3(V)

lp_wrapper1()
lp_wrapper2()
lp_wrapper3()

lprofiler1.print_stats()
lprofiler2.print_stats()
lprofiler3.print_stats()

4.4544402182900615
4.4544402182900615
4.4544402182900615
Timer unit: 1e-09 s

Total time: 1.02577 s
File: <ipython-input-17-b2bb0de7e554>
Function: A at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def A():
     3         1      13096.0  13096.0      0.0    a1 = []
     4     12094  893823470.0  73906.4     87.1    for i, row in rev[(rev['date']<"2011-01-01") & (rev["date"]>= "2010-01-01")].iterrows():
     5     12094  131255687.0  10853.0     12.8      a1.append(row["rating"])
     6         1     674563.0 674563.0      0.1    print(sum(a1)/len(a1))

Timer unit: 1e-09 s

Total time: 0.971726 s
File: <ipython-input-17-b2bb0de7e554>
Function: B at line 8

Line #      Hits         Time  Per Hit   % Time  Line Contents
     8                                           def B():
     9         1   19995630.0 19995630.0      2.1    revr = rev[(rev['date']<"2011-01-01") & (rev["date"]>= "2010-01-01")]
    10         1 

In [ ]:
def B():
  a = [i for i in rev[(rev['date']<"2011-01-01") & (rev["date"]>= "2010-01-01")]["rating"]]
  print(sum(a)/len(a))

lprofiler2 = LineProfiler()

lp_wrapper2 = lprofiler2(B)

lp_wrapper2()

lprofiler2.print_stats()


4.4544402182900615
Timer unit: 1e-09 s

Total time: 0.0325459 s
File: <ipython-input-57-6f6e052d009a>
Function: B at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def B():
     2         1   31669028.0 31669028.0     97.3    a = [i for i in rev[(rev['date']<"2011-01-01") & (rev["date"]>= "2010-01-01")]["rating"]]
     3         1     876896.0 876896.0      2.7    print(sum(a)/len(a))



3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [ ]:
df = pd.read_csv("reviews_sample.csv")
df

,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126692,158736,2282344,8701,2012-06-03,0,This recipe is outstanding. I followed the rec...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [ ]:
df = pd.read_csv("reviews_sample.csv")
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

lprofiler = LineProfiler()

lp_wrapper = lprofiler(get_word_reviews_count)

lp_wrapper(df)

lprofiler.print_stats()

Timer unit: 1e-09 s

Total time: 46.5969 s
File: <ipython-input-69-afff0cf1321a>
Function: get_word_reviews_count at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def get_word_reviews_count(df):
     3         1        985.0    985.0      0.0      word_reviews = {}
     4    126679 12182806085.0  96170.7     26.1      for _, row in df.dropna(subset=['review']).iterrows():
     5    126679 2806348017.0  22153.2      6.0          recipe_id, review = row['recipe_id'], row['review']
     6    126679  680653245.0   5373.1      1.5          words = review.split(' ')
     7   6792010 1718311802.0    253.0      3.7          for word in words:
     8   6617066 3214235750.0    485.7      6.9              if word not in word_reviews:
     9    174944   91945440.0    525.6      0.2                  word_reviews[word] = []
    10   6792010 4482041403.0    659.9      9.6              word_reviews[word].append(recipe_id)
    11

In [ ]:
df = pd.read_csv("reviews_sample.csv")
def get_word_reviews_count(df):
    word_reviews = {}
    for i in df["review"]:
        for word in str(i).split(" "):
            if word in word_reviews:
              word_reviews[word] += 1
            else:
              word_reviews[word] = 1
    return word_reviews


lprofiler = LineProfiler()

lp_wrapper = lprofiler(get_word_reviews_count)

lp_wrapper(df)

lprofiler.print_stats()

Timer unit: 1e-09 s

Total time: 8.02315 s
File: <ipython-input-18-17f105368fb3>
Function: get_word_reviews_count at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           def get_word_reviews_count(df):
     3         1       1162.0   1162.0      0.0      word_reviews = {}
     4    126696   67850787.0    535.5      0.8      for i in df["review"]:
     5   6792027 2147449919.0    316.2     26.8          for word in str(i).split(" "):
     6   6617083 2636721932.0    398.5     32.9              if word in word_reviews:
     7   6617083 3090135479.0    467.0     38.5                word_reviews[word] += 1
     8                                                       else:
     9    174944   80994392.0    463.0      1.0                word_reviews[word] = 1
    10         1        869.0    869.0      0.0      return word_reviews



4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [ ]:
import pandas as pd

df = pd.read_csv('reviews_sample.csv')
df= df[df['rating'] != 0]
df

,Unnamed: 0,user_id,recipe_id,date,rating,review
0,370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
1,624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
2,187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
3,706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
4,312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...
...,...,...,...,...,...,...
126690,344676,724631,314698,2008-10-14,5,"5 Stars, The Hunter said the mushrooms and sau..."
126691,1013457,1270706,335534,2009-05-17,4,This recipe was great! I made it last night. I...
126693,1059834,689540,222001,2008-04-08,5,"Well, we were not a crowd but it was a fabulou..."
126694,453285,2000242659,354979,2015-06-02,5,I have been a steak eater and dedicated BBQ gr...


In [ ]:
#Без использования векторизованных операций и методов массивов numpy и без использования numba
a = df["recipe_id"].unique()
srznach = {}
for i in a:
   srznach[i] = df["rating"].where(df["recipe_id"]== i).mean()
print(srznach)   


#### [версия 2]
* Уточнены формулировки задач 1, 3, 4